<a href="https://colab.research.google.com/github/David-Mwatha/ADS-Assignment-1/blob/main/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving take-at-home.ipynb to take-at-home.ipynb


In [ ]:
!pip install tensorflow


In [ ]:
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.3 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.1
    Uninstalling ml-dtypes-0.3.1:
      Successfully uninstalled ml-dtype

In [1]:
import tensorflow as tf
import numpy as np

# Define data preprocessing and filtering
num_classes = 9
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'truck']
(x_train, y_train), (_, _) = tf.keras.datasets.cifar10.load_data()

# Filter out the classes that are not in the list of class names
valid_classes = [class_names.index(class_name) for class_name in class_names]

# Initialize empty lists for filtered data
filtered_x_train = []
filtered_y_train = []

for i in range(len(y_train)):
    label = y_train[i][0]
    if label in valid_classes:
        filtered_x_train.append(x_train[i])
        filtered_y_train.append(valid_classes.index(label))

x_train = np.array(filtered_x_train)
y_train = tf.keras.utils.to_categorical(filtered_y_train, num_classes)

# Normalize the data
x_train = x_train / 255.0


170498071/170498071 [==============================] - 2s 0us/step


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, Model

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Modify the model for the 9-class classification task
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Set the layers of the base model to be non-trainable
for layer in base_model.layers:
    layer.trainable = False


94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=64)


Epoch 1/10
704/704 [==============================] - 212s 294ms/step - loss: 2.0071 - accuracy: 0.2657
Epoch 2/10
704/704 [==============================] - 210s 299ms/step - loss: 1.8165 - accuracy: 0.3267
Epoch 3/10
704/704 [==============================] - 206s 292ms/step - loss: 1.7624 - accuracy: 0.3487
Epoch 4/10
704/704 [==============================] - 203s 289ms/step - loss: 1.7372 - accuracy: 0.3589
Epoch 5/10
704/704 [==============================] - 207s 294ms/step - loss: 1.7033 - accuracy: 0.3752
Epoch 6/10
704/704 [==============================] - 204s 290ms/step - loss: 1.6868 - accuracy: 0.3825
Epoch 7/10
704/704 [==============================] - 203s 288ms/step - loss: 1.6682 - accuracy: 0.3894
Epoch 8/10
704/704 [==============================] - 208s 295ms/step - loss: 1.6566 - accuracy: 0.3958
Epoch 9/10
704/704 [==============================] - 204s 290ms/step - loss: 1.6416 - accuracy: 0.4008
Epoch 10/10
704/704 [==============================] - 204s 291m

In [2]:
import tensorflow.keras as keras
model = keras.models.Sequential()
model.add(keras.layers.Dense(64, activation='relu'))

from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Create a submodel to access intermediate activations
layer_outputs = [layer.output for layer in model.layers]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

# Select a sample image
sample_image = x_train[0:1]

# Get intermediate activations
activations = activation_model.predict(sample_image)

# Visualize intermediate activations
for layer_name, layer_activation in zip([layer.name for layer in model.layers], activations):
    if 'conv1' in layer_name or 'res4' in layer_name or 'res5' in layer_name:
        n_features = layer_activation.shape[-1]
        size = layer_activation.shape[1]
        n_cols = n_features // 8
        display_grid = np.zeros((size * n_cols, 8 * size))

        for col in range(n_cols):
            for row in range(8):
                channel_image = layer_activation[0, :, :, col * 8 + row]
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image

        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')

plt.show()


AttributeError: ignored

In [ ]:
# part 2

In [ ]:
# Import the necessary libraries
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# Define a modified ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Add custom classification layers for the "ship" class
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the fine-tuning model
model_finetune = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model_finetune.compile(optimizer=SGD(lr=0.001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the model for "ship" class
model_finetune.fit(x_train, y_train_ship, epochs=10, batch_size=64)


In [ ]:
# Create a feature extraction model
model_feat_extract = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

# Extract features for the "ship" class
features_ship = model_feat_extract.predict(x_train)


In [ ]:
# Visualization before fine-tuning
# Replace model_finetune with model_feat_extract for visualization after feature extraction

import matplotlib.pyplot as plt
import numpy as np

def visualize_activations(model, sample_image, layer_name, figsize=(12, 8)):
    # Create a submodel to access intermediate activations
    layer_outputs = [layer.output for layer in model.layers]
    activation_model = Model(inputs=model.input, outputs=layer_outputs)

    # Get intermediate activations
    activations = activation_model.predict(sample_image)

    # Visualize intermediate activations
    plt.figure(figsize=figsize)
    for i, (layer_name, layer_activation) in enumerate(zip([layer.name for layer in model.layers], activations)):
        if 'conv1' in layer_name or 'conv3_block4_out' in layer_name:
            n_features = layer_activation.shape[-1]
            size = layer_activation.shape[1]
            n_cols = n_features // 8
            display_grid = np.zeros((size * n_cols, 8 * size))

            for col in range(n_cols):
                for row in range(8):
                    channel_image = layer_activation[0, :, :, col * 8 + row]
                    channel_image -= channel_image.mean()
                    channel_image /= channel_image.std()
                    channel_image *= 64
                    channel_image += 128
                    channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                    display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image

            scale = 1. / size
            plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
            plt.title(f'Activation of {layer_name}')
            plt.grid(False)
            plt.imshow(display_grid, aspect='auto', cmap='viridis')

# Visualize activations before fine-tuning
visualize_activations(model_finetune, sample_image, layer_name='conv1', figsize=(12, 8))

# Visualize activations after fine-tuning
# Replace model_finetune with model_feat_extract for visualization after feature extraction
visualize_activations(model_finetune, sample_image, layer_name='conv1', figsize=(12, 8))

plt.show()


NameError: ignored